In [2]:
import win32gui, win32ui, win32con
from time import sleep
import cv2
import pyautogui
import numpy as np
from PIL import ImageGrab
import pytesseract
import PIL

## lista janelas 

In [3]:
def list_windows_name():
    def winEnumHandler(hwnd,ctx):
        if win32gui.IsWindowVisible(hwnd):
            print(hex(hwnd), win32gui.GetWindowText(hwnd) )
    win32gui.EnumWindows(winEnumHandler,None)    

In [4]:
list_windows_name()

0x20040 
0x100a2 
0x100e4 
0x20572 ● main.ipynb - learn_win_api - Visual Studio Code
0x6009a Electronic - Listen to Free Radio Stations - AccuRadio - Google Chrome
0x10690 terminal.png - IrfanView (Zoom: 1029 x 650)
0x10388 Configurações
0x30372 Configurações
0x402f8 Experiência de Entrada do Windows
0x1017a 
0x1018c 
0x3004c 
0x1015a 
0x1014e 
0x1014c 
0x1014a 
0x10148 
0x10110 Program Manager


## procura por janelas filhas

In [5]:
def get_inner_windows(whndl):
    def callback(hwnd, hwnds):
        if win32gui.IsWindowVisible(hwnd) and win32gui.IsWindowEnabled(hwnd):
            hwnds[win32gui.GetClassName(hwnd)] = hwnd
        return True
    hwnds = {} 
    win32gui.EnumChildWindows(whndl, callback, hwnds)
    return hwnds

## interage com janela

In [6]:
def interact():
    window_name = 'teste - wc3270'
    hwnd = win32gui.FindWindow(None, window_name)
    # hwnd =  get_inner_windows(hwnd)['Edit']
    win = win32ui.CreateWindowFromHandle(hwnd)
    
    # win.SendMessage(win32con.WM_KEYDOWN, 0x32, 0)
    # sleep(0.01)
    # win.SendMessage(win32con.WM_KEYUP, 0x32, 0)

    win.SendMessage(win32con.WM_CHAR, ord('A'), 0)

## Captura

In [7]:

def window_captura():

        # w = 1040 # set this
        # h = 720 # set this
        # bmpfilenamename = "out.bmp" #set this
    
    hwnd = win32gui.FindWindow(None, 'terminal.png - IrfanView (Zoom: 1029 x 650)')



    window_rec = win32gui.GetWindowRect(hwnd)
    w = window_rec[2] - window_rec[0]
    h = window_rec[3] - window_rec[1]

    border_pixel = 8
    titlebar_pixel = 30
    w = w -     (border_pixel * 2)
    h = h - titlebar_pixel - border_pixel


    wDC = win32gui.GetWindowDC(hwnd)
    dcObj=win32ui.CreateDCFromHandle(wDC)
    cDC=dcObj.CreateCompatibleDC()
    dataBitMap = win32ui.CreateBitmap()
    dataBitMap.CreateCompatibleBitmap(dcObj, w, h)
    cDC.SelectObject(dataBitMap)
    cDC.BitBlt((0,0),(w, h) , dcObj, (0,0), win32con.SRCCOPY)
    
    # # save image
    # dataBitMap.SaveBitmapFile(cDC, 'dubug.bmp')

    singedIntsArray = dataBitMap.GetBitmapBits(True)
    img = np.fromstring(singedIntsArray,dtype='uint8')
    img.shape = (h,w,4)

    # Free Resources
    dcObj.DeleteDC()
    cDC.DeleteDC()
    win32gui.ReleaseDC(hwnd, wDC)
    win32gui.DeleteObject(dataBitMap.GetHandle())

    return img

In [8]:
while(True):

    screenshot = window_captura()
    # screenshot = np.array(screenshot)
    # screenshot = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)
   
    cv2.imshow('computer vision',screenshot)

    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        break


C:\Users\Junior\AppData\Local\Temp\ipykernel_15104\3659337590.py:33: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(singedIntsArray,dtype='uint8')


## OCR 
 -> [download link](https://digi.bib.uni-mannheim.de/tesseract/tesseract-ocr-w64-setup-v5.0.1.20220118.exe)

In [15]:
filename = 'terminal.png'
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\...\Tesseract-OCR\tesseract.exe'
img1 = np.array(PIL.Image.open(filename))
text = pytesseract.image_to_string(img1)

In [17]:
print(text)

DSN = VENDOR.VTAM.SOURCE(USSTAB3) 2014-08-05 16:29:10.803732

==WELCOME==TO THE=

IBM Innovation Center - Dallas==

Net Name

LU Name TCPO0O709

SSCP Name

Client IP 10.32.24.69
Client Name

Date/Time 05/04/17 15:18:17

Choese from the following commands Tso
cics
CICSVBIA
JERRY

TSO on Dalia
CICSV41A (4.1)
CICSVBIA (3.2)
z/VM on Jerry

A mysessio

